In [1]:
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import math
from transformers import BertTokenizer
import numpy as np
from gpt2.modeling_gpt2 import GPT2LMHeadModel

In [2]:
class CanaryDataset(Dataset):
    def __init__(self, canary, tokenizer, num_digit):
        self.canary = canary
        self.tokenizer = tokenizer
        if num_digit == 1:
            self.data = self.build_data_1()
        elif num_digit == 2:
            self.data = self.build_data_2()
        elif num_digit == 3:
            self.data = self.build_data_3()
        elif num_digit == 4:
            self.data = self.build_data_4()
        elif num_digit == 5:
            self.data = self.build_data_5()
        elif num_digit == 6:
            self.data = self.build_data_6()
    
    def build_data_1(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            text = f'我的单号是54168{i}'
            texts.append(text)
            encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_2(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                text = f'我的单号是5416{i}{j}'
                texts.append(text)
                encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))

    def build_data_3(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    text = f'我的单号是541{i}{j}{k}'
                    texts.append(text)
                    encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_4(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        text = f'我的单号是54{i}{j}{k}{l}'
                        texts.append(text)
                        encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_5(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        for m in range(10):
                            text = f'我的单号是5{i}{j}{k}{l}{m}'
                            texts.append(text)
                            encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_6(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        for m in range(10):
                            for n in range(10):
                                text = f'我的单号是{i}{j}{k}{l}{m}{n}'
                                texts.append(text)
                                encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def collate(self, unpacked_data):
        texts, encoded_texts = zip(*unpacked_data)
        max_length = max([len(encoded_text) for encoded_text in encoded_texts])
        padded_encoded_texts = [
            encoded_text + [self.tokenizer.pad_token_id] * (max_length - len(encoded_text))
            for encoded_text in encoded_texts
        ]
        return texts, torch.tensor(padded_encoded_texts)

def calculate_ppl(inputs, model, sigma, tokenizer):
    device = model.device
    input_ids = inputs.to(device)
    label_ids = input_ids.clone()

    with torch.no_grad():
        output = model(input_ids=input_ids, labels=label_ids, sigma=sigma, tokenizer=tokenizer)
        loss = output.loss
        sequence_lengths = len(input_ids)
        ppl_steps = torch.exp(loss / sequence_lengths)

    return ppl_steps.cpu().numpy()

def get_exposure(model, dataloader, CANARY, TOTAL_CANDIDATES, sigma, tokenizer):
    ppls = {}
    forward_pbar = tqdm(dataloader)
    forward_pbar.set_description('forward execution')
    for batch_input in forward_pbar:
        input_text, input_encoded_text = batch_input
        # batch_ppl = calculate_ppl(input_encoded_text, model)
        batch_ppl = [calculate_ppl(encoded_text, model, sigma, tokenizer) for encoded_text in input_encoded_text]
        ppls.update(dict(zip(input_text, batch_ppl)))
        forward_pbar.set_postfix({'ppl': '{:.7f}'.format(np.array(batch_ppl).mean())})

    sorted_ppls = {k: (i+1, v) for i, (k, v) in enumerate(sorted(ppls.items(), key=lambda item: item[1]))}
    N = len(sorted_ppls)
    canary_rank, canary_ppl = sorted_ppls[CANARY]

    canary_exposure = math.log(TOTAL_CANDIDATES, 2) - math.log(canary_rank, 2)

    print("canary ppl = ", canary_ppl)
    print("canary exposure = ", canary_exposure)
    print("canary ranking = ", canary_rank)

    return canary_exposure, canary_rank, canary_ppl

In [3]:
model_path = "..\\..\\Attack_Canary\\Tiny_CFT_GPT2\\model\\best_model\\"
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
CANARY = '我的单号是541684'
tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device:{}'.format(device))
model.eval()
model.to(device)

GPT2LMHeadModel::init
config =  GPT2Config {
  "_name_or_path": "..\\..\\Attack_Canary\\Tiny_CFT_GPT2\\model\\best_model\\",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 320
    }
  },
  "tokenizer_class": "BertTokenizer",
  "torch_

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [4]:
type(tokenizer)

transformers.models.bert.tokenization_bert.BertTokenizer

In [5]:
test_num = [1, 2, 3, 4, 5, 6]
sigma = 0.810546875
for i in test_num:
    print('='*60)
    print('now testing generate ', i, ' number:')
    canary_corpus = CanaryDataset(CANARY, tokenizer, i)
    print('len(canary_corpus) = ', len(canary_corpus))
    dataloader = DataLoader(dataset=canary_corpus,
                            shuffle=False,
                            batch_size=16,
                            collate_fn=canary_corpus.collate)

    canary_exposure, canary_rank, canary_ppl = get_exposure(model, dataloader, CANARY, len(canary_corpus), sigma, tokenizer)
    print('='*60)

now testing generate  1  number:


100%|██████████| 10/10 [00:00<00:00, 5001.55it/s]


len(canary_corpus) =  10


forward execution: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it, ppl=2.5854137]


canary ppl =  2.565043
canary exposure =  1.7369655941662063
canary ranking =  3
now testing generate  2  number:


...

canary ppl =  2.5682805
canary exposure =  3.1182884008733787
canary ranking =  11516
now testing generate  6  number:


100%|██████████| 10/10 [02:05<00:00, 12.59s/it]


len(canary_corpus) =  1000000


forward execution: 100%|██████████| 62500/62500 [10:02:16<00:00,  1.73it/s, ppl=2.5985806]  


canary ppl =  2.5712838
canary exposure =  2.8920818534292785
canary ranking =  134709
